# Select Your IPTS 

In [1]:
# #from __code.filename_metadata_match import FilenameMetadataMatch

# from __code import system
# system.System.select_working_dir()
# from __code.__all import custom_style
# custom_style.style()

In [23]:
import ipywe.fileselector
from IPython.core.display import HTML
from __code.time_utility import RetrieveTimeStamp
import os

class FilenameMetadataMatch(object):

    data_folder = ''
    metadata_file = ''
    
    list_data_time_stamp = None
    
    def __init__(self, working_dir='./'):
        self.working_dir = working_dir

    def select_input_folder(self):
        _instruction = "Select Input Folder ..."
        self.input_folder_ui = ipywe.fileselector.FileSelectorPanel(instruction=_instruction,
                                                                    start_dir=self.working_dir,
                                                                    next=self.select_input_folder_done,
                                                                    type='directory',
                                                                   )
        self.input_folder_ui.show()
        
    def select_input_folder_done(self, folder):
        self.data_folder = folder
        display(HTML('Folder Selected: <span style="font-size: 20px; color:green">' + folder))

    def select_metadata_file(self):
        _instruction = "Select Metadata File ..."
        self.metadata_ui = ipywe.fileselector.FileSelectorPanel(instruction=_instruction,
                                                                start_dir=self.working_dir,
                                                                next=self.select_metadata_file_done,
                                                               )
        self.metadata_ui.show()
        
    def select_metadata_file_done(self, metadata_file):
        self.metadata_file = metadata_file
        display(HTML('Metadata File Selected: <span style="font-size: 20px; color:green">' + metadata_file))
        
    def retrieve_time_stamp(self):
        o_retriever = RetrieveTimeStamp(folder=self.data_folder)
        o_retriever._run()
        self.list_data_time_stamp = o_retriever
    
    def load_metadata(self):
        metadata_file = self.metadata_file
    

# Select Input Folder 

In [3]:
#o_match = FilenameMetadataMatch(working_dir=system.System.get_working_dir())
o_match = FilenameMetadataMatch(working_dir='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-20444-Regina/')
o_match.select_input_folder()

VBox(children=(Label(value='Select Input Folder ...', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-20444-Regina', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), Select(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                   |     Fri Sep 28 15:18:15 2018', ' Sep_17_normalized_part1     |     Directory', ' metadata_files              |     Directory', ' test_output                 |     Directory'), value=' .')), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

# Select Metadata File 

In [4]:
o_match.select_metadata_file()

VBox(children=(Label(value='Select Metadata File ...', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-20444-Regina', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), Select(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                   |     Fri Sep 28 15:18:15 2018', ' Sep_17_normalized_part1     |     Directory', ' metadata_files              |     Directory', ' test_output                 |     Directory'), value=' .')), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

# Retrieve Time Stamp

In [5]:
o_match.retrieve_time_stamp()

# Load Metadata File

# DEBUGGING STARTS HERE

In [10]:
import pandas as pd
import codecs
from __code.file_handler import get_file_extension
from ipywidgets import widgets
import os

In [16]:
#metadata_file = o_match.metadata_file
import glob
metadata_list_files = glob.glob('/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-20444-Regina/metadata_files_txt/metadata*.mpt.txt')
index_file = 0

metadata_file = metadata_list_files[index_file]



assert os.path.exists(metadata_file)

In [61]:
class Metadata():
    
    reference_line_showing_end_of_metadata = 'Number of loops'
    end_of_metadata_after_how_many_lines_from_reference_line = 2

    nbr_row_metadata = -1
    
    def __init__(self, filename=''):
        self.filename = filename
        
    def calculate_nbr_row_metadata(self):
        file_handler = codecs.open(self.filename,
                                   'r',
                                   encoding='utf_8',
                                   errors='ignore')
        
        for _row_index, _row in enumerate(file_handler.readlines()):
            if self.reference_line_showing_end_of_metadata in _row:
                self.nbr_row_metadata = _row_index + self.end_of_metadata_after_how_many_lines_from_reference_line + 1


class MPTFileParser(object):
    
    nbr_row_metadata = -1
    
    metadata_dict = {'Acquisition started on': {"split1": ' : ',
                                                "value": '',
                                                "split2": '',
                                               },
                     'Electrode surface area': {"split1": ' : ',
                                                "value": '',
                                                "split2": ' ',
                                                "units": '',
                                               },                     
                    }
        
    time_info_index_column = 2 # in this file, the time information is in the column #2

    def __init__(self, filename=''):
        self.filename = filename
        self.evaluate_nbr_row_metadata()
        self.parse()
    
    def evaluate_nbr_row_metadata(self):
        o_metadata = Metadata(filename=self.filename)
        o_metadata.calculate_nbr_row_metadata()
        self.nbr_row_metadata = o_metadata.nbr_row_metadata
    
    def parse(self):
        self.read_data()
        self.read_metadata()
        
    def read_data(self):
        o_pd = pd.read_csv(self.filename,
                           sep='\t',
                           skiprows=self.nbr_row_metadata,
                          )
        self.o_pd = o_pd
        
    def read_metadata(self):
        fdata = codecs.open(self.filename,
                           'r',
                           encoding='utf_8',
                           errors='ignore')
        metadata = []
        for _row in np.arange(self.nbr_row_metadata):
            metadata.append(fdata.readline())
        self.metadata = metadata

        self.read_specific_metadata()
        
    def read_specific_metadata(self):
        for _keys in self.metadata_dict.keys():
            for _line in self.metadata:
                if _keys in _line:
                    result = _line.split(self.metadata_dict[_keys]['split1']) #1st split
                    if not self.metadata_dict[_keys]['split2']:
                        self.metadata_dict[_keys]['value'] = result[1].strip()
                    else: #2nd split
                        [value, units] = result[1].strip().split(self.metadata_dict[_keys]['split2'])
                        self.metadata_dict[_keys]['value'] = value.strip()
                        self.metadata_dict[_keys]['units'] = units.strip()

                
class MetadataFileParser(object):
    
    filename = ''
    meta_type = ''
    
    def __init__(self, filename='', meta_type=''):
        self.filename = filename
        if meta_type:
            self.meta_type = meta_type
        else:
            self.meta_type = get_file_extension(filename)
                        
    def parse(self):
        if self.meta_type == 'mpt':
            o_mpt = MPTFileParser(filename=self.filename)
            self.meta = o_mpt
        else:
            raise NotImplementedError("This file format is not supported!")
        
    def create_metadata_selected_vs_time_stamp(self, metadata_selected):
        #time_column = self.meta.o_pd.
        pass
        

        
        

In [62]:
o_meta = MetadataFileParser(filename=metadata_file, meta_type='mpt')
o_meta.parse()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb5 in position 3: invalid start byte

In [55]:
# o_meta.meta.metadata

In [56]:
o_meta.meta.metadata_dict

{'Acquisition started on': {'split1': ' : ',
  'split2': '',
  'value': '09/12/2018 23:08:58'},
 'Electrode surface area': {'split1': ' : ',
  'split2': ' ',
  'units': 'cm',
  'value': '0.001'}}

In [59]:
#o_meta.filename

In [60]:
o_meta.meta.o_pd

,1.0000186E+006,-5.7711523E+003,1.4303259E+004,1.5423664E+004,-1.1197334E+002,4.179466011846671E+000,7.8103616E-004,3.5815124E-004,1.1126974E-005,9.5691184E-006,1.000000000000000E+000,12,4.2146053E-003,2.7325578E-007,-2.4259813E-005,6.0125665E-005,6.4835440E-005,1.1197334E+002
0,856433.690000,1221.8723,3487.05250,3694.9299,-70.689407,5.006456,-0.000703,-0.000703,0.000053,0.000047,1.0,13,0.005716,0.000002,NaN,NaN,NaN,NaN
1,733475.000000,1667.0239,4157.11720,4478.9053,-68.148979,5.543479,-0.001451,-0.001451,0.000052,0.000045,1.0,13,0.009564,0.000002,NaN,NaN,NaN,NaN
2,628173.250000,2260.4304,4707.05960,5221.6812,-64.348686,6.081478,-0.001774,-0.001774,0.000054,0.000044,1.0,13,0.014838,0.000003,NaN,NaN,NaN,NaN
3,537986.190000,2909.6228,5143.77780,5909.6831,-60.504951,6.618465,-0.001936,-0.001936,0.000058,0.000044,1.0,13,0.021542,0.000004,NaN,NaN,NaN,NaN
4,460742.660000,3564.4856,5506.34380,6559.3735,-57.083336,7.155476,-0.002102,-0.002102,0.000063,0.000044,1.0,13,0.029596,0.000005,NaN,NaN,NaN,NaN
5,394596.190000,4272.0137,5953.93800,7327.9927,-54.340160,7.693462,-0.002316,-0.002316,0.000068,0.000045,1.0,13,0.039086,0.000005,NaN,NaN,NaN,NaN
6,337941.660000,5061.6069,6319.53560,8096.6904,-51.307156,8.231462,-0.002467,-0.002467,0.000075,0.000045,1.0,13,0.049935,0.000006,NaN,NaN,NaN,NaN
7,289424.380000,5922.3521,6604.59810,8871.0186,-48.117378,8.768458,-0.002553,-0.002553,0.000083,0.000046,1.0,13,0.062201,0.000007,NaN,NaN,NaN,NaN
8,247875.220000,6796.9604,6804.84810,9617.9326,-45.033226,9.173478,-0.002635,-0.002635,0.000094,0.000047,1.0,13,0.075941,0.000008,NaN,NaN,NaN,NaN
9,212282.880000,7806.8818,6851.75630,10387.2020,-41.271999,9.711464,-0.002642,-0.002642,0.000109,0.000048,1.0,13,0.080389,0.000008,NaN,NaN,NaN,NaN


In [52]:
list_columns = list(o_meta.meta.o_pd.columns.values)
#list_columns.pop(o_meta.meta.time_info_index_column)
print(list_columns)

['1.0000186E+006', '-5.7711523E+003', '1.4303259E+004', '1.5423664E+004', '-1.1197334E+002', '4.179466011846671E+000', '7.8103616E-004', '3.5815124E-004', '1.1126974E-005', '9.5691184E-006', '1.000000000000000E+000', '12', '4.2146053E-003', '2.7325578E-007', '-2.4259813E-005', '6.0125665E-005', '6.4835440E-005', '1.1197334E+002']


In [8]:
box = widgets.HBox([widgets.Label("Select Metadata:",
                                layout=widgets.Layout(width='15%'),
                                ),
                   widgets.Dropdown(options=list_columns,
                                   value=list_columns[0],
                                   layout=widgets.Layout(width='30%')),
                   ])
display(box)
                   

HBox(children=(Label(value='Select Metadata:', layout=Layout(width='15%')), Dropdown(layout=Layout(width='30%'), options=('mode', 'error', 'Ewe/V', 'Unnamed: 4'), value='mode')))

In [9]:
metadata_selected = box.children[1].value
metadata_selected

'Ewe/V'